In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

1. Import claims_data.csv and cust_data.csv which is provided to you and
combine the two datasets appropriately to create a 360-degree view of
the data. Use the same for the subsequent questions.

In [3]:
claims_df = pd.read_csv('claims.csv')
cust_demographics_df = pd.read_csv('cust_demographics.csv')

In [4]:
claims_df

,claim_id,customer_id,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent
0,54004764,21868593,Driver error,11/27/2017,Auto,No,Material only,$2980,1.0,No
1,33985796,75740424,Crime,10/03/2018,Home,Unknown,Material only,$2980,3.0,No
2,53522022,30308357,Other driver error,02/02/2018,Auto,No,Material only,$3369.5,1.0,Yes
3,13015401,47830476,Natural causes,06/17/2018,Auto,No,Material only,$1680,1.0,No
4,22890252,19269962,Crime,01/13/2018,Auto,No,Material only,$2680,1.0,No
...,...,...,...,...,...,...,...,...,...,...
1095,97727122,35951012,Other driver error,06/11/2017,Auto,No,Material only,$3059,4.0,Yes
1096,10247193,14818669,Natural causes,03/14/2018,Auto,No,Material only,$1520,2.0,No
1097,79807493,85322831,Other driver error,02/09/2018,Auto,No,Material and injury,$23575,2.0,Yes
1098,69299345,73449366,Other causes,03/21/2018,Auto,Unknown,Material and injury,$25120,1.0,No


In [5]:
cust_demographics_df

,CUST_ID,gender,DateOfBirth,State,Contact,Segment
0,21868593,Female,12-Jan-79,VT,789-916-8172,Platinum
1,75740424,Female,13-Jan-70,ME,265-543-1264,Silver
2,30308357,Female,11-Mar-84,TN,798-631-4758,Silver
3,47830476,Female,01-May-86,MA,413-187-7945,Silver
4,19269962,Male,13-May-77,NV,956-871-8691,Gold
...,...,...,...,...,...,...
1080,79539873,Female,15-Mar-81,ND,459-425-4319,Platinum
1081,42364152,Female,07-Jul-96,ID,529-462-1635,Silver
1082,19888166,Male,11-Apr-90,WI,712-651-9613,Gold
1083,11256802,Female,22-Oct-64,LA,469-345-5617,Silver


In [7]:
merged_df = pd.merge(claims_df, cust_demographics_df, left_on='customer_id', right_on='CUST_ID', how='left')
merged_df['claim_amount'] = merged_df['claim_amount'].replace('[\$,]', '', regex=True).astype(float)
merged_df['unreported_injury_claim'] = merged_df['police_report'].apply(lambda x: 1 if x == 'No' else 0)

merged_df['DateOfBirth'] = pd.to_datetime(merged_df['DateOfBirth'], format='%d-%b-%y', errors='coerce')
merged_df['age'] = (pd.to_datetime('today') - merged_df['DateOfBirth']).dt.days // 365.25

def categorize_age(age):
    if age < 18:
        return 'Children'
    elif 18 <= age < 30:
        return 'Youth'
    elif 30 <= age < 60:
        return 'Adult'
    else:
        return 'Senior'

merged_df['age_category'] = merged_df['age'].apply(categorize_age)

merged_df['claim_amount'].fillna(merged_df['claim_amount'].mean(), inplace=True)
merged_df['total_policy_claims'].fillna(merged_df['total_policy_claims'].mode()[0], inplace=True)
merged_df.head()

,claim_id,customer_id,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,CUST_ID,gender,DateOfBirth,State,Contact,Segment,unreported_injury_claim,age,age_category
0,54004764,21868593,Driver error,11/27/2017,Auto,No,Material only,2980.0,1.0,No,21868593.0,Female,1979-01-12,VT,789-916-8172,Platinum,1,45.0,Adult
1,33985796,75740424,Crime,10/03/2018,Home,Unknown,Material only,2980.0,3.0,No,75740424.0,Female,1970-01-13,ME,265-543-1264,Silver,0,54.0,Adult
2,53522022,30308357,Other driver error,02/02/2018,Auto,No,Material only,3369.5,1.0,Yes,30308357.0,Female,1984-03-11,TN,798-631-4758,Silver,1,40.0,Adult
3,13015401,47830476,Natural causes,06/17/2018,Auto,No,Material only,1680.0,1.0,No,47830476.0,Female,1986-05-01,MA,413-187-7945,Silver,1,38.0,Adult
4,22890252,19269962,Crime,01/13/2018,Auto,No,Material only,2680.0,1.0,No,19269962.0,Male,1977-05-13,NV,956-871-8691,Gold,1,47.0,Adult


2. Perform a data audit for the datatypes and find out if there are any mismatch within the current datatypes of the columns and their business significance.

In [11]:
print(combined_df.dtypes)

claim_id                            int64
customer_id                         int64
incident_cause                     object
claim_date                         object
claim_area                         object
police_report                      object
claim_type                         object
claim_amount                      float64
total_policy_claims               float64
fraudulent                         object
CUST_ID                           float64
gender                             object
DateOfBirth                datetime64[ns]
State                              object
Contact                            object
Segment                            object
unreported_injury_claim             int64
age                               float64
age_category                       object
dtype: object

In [16]:
print("Data Types of Columns:")
print(merged_df.dtypes)

expected_data_types = {
    "customer_id": "int64",
    "claim_id": "int64",
}

mismatches = []
for column, expected_type in expected_data_types.items():
    actual_type = merged_df[column].dtype
    if actual_type != expected_type:
        mismatches.append((column, expected_type, actual_type))

if mismatches:
    print("\nData Type Mismatches:")
    for column, expected_type, actual_type in mismatches:
        print(f"Column: {column}, Expected Type: {expected_type}, Actual Type: {actual_type}")
else:
    print("\nNo Data Type Mismatches Found.")

Data Types of Columns:
claim_id                            int64
customer_id                         int64
incident_cause                     object
claim_date                         object
claim_area                         object
police_report                      object
claim_type                         object
claim_amount                      float64
total_policy_claims               float64
fraudulent                         object
CUST_ID                           float64
gender                             object
DateOfBirth                datetime64[ns]
State                              object
Contact                            object
Segment                            object
unreported_injury_claim             int64
age                               float64
age_category                       object
dtype: object

No Data Type Mismatches Found.


In [18]:
merged_df = pd.merge(claims_df, cust_demographics_df, left_on='customer_id', right_on='CUST_ID', how='left')

# Convert claim_amount to numeric by removing the $ sign
merged_df['claim_amount'] = merged_df['claim_amount'].replace('[\$,]', '', regex=True).astype(float)

# Display the first few rows of the updated DataFrame to confirm the changes
print(merged_df[['claim_amount']].head())


   claim_amount
0        2980.0
1        2980.0
2        3369.5
3        1680.0
4        2680.0


In [20]:
# Create an alert flag for unreported injury claims
merged_df['unreported_injury_claim'] = merged_df.apply(
    lambda row: 1 if row['claim_type'] in ['Injury only', 'Material and injury'] and row['police_report'] == 'No' else 0, 
    axis=1
)

# Display the first few rows to confirm the changes
print(merged_df[['claim_type', 'police_report', 'unreported_injury_claim']].head())


      claim_type police_report  unreported_injury_claim
0  Material only            No                        0
1  Material only       Unknown                        0
2  Material only            No                        0
3  Material only            No                        0
4  Material only            No                        0


In [21]:

merged_df['claim_date'] = pd.to_datetime(merged_df['claim_date'], format='%d-%b-%y', errors='coerce')

# Sort by customer_id and claim_date in descending order
merged_df.sort_values(by=['customer_id', 'claim_date'], ascending=[True, False], inplace=True)

# Drop duplicates, keeping only the most recent claim for each customer_id
unique_customers_df = merged_df.drop_duplicates(subset='customer_id', keep='first')

# Display the first few rows of the unique customers DataFrame to confirm the changes
print(unique_customers_df.head())


     claim_id  customer_id      incident_cause claim_date claim_area  \
184  69348631       154557        Driver error        NaT       Auto   
836  40953049       263204        Other causes        NaT       Auto   
677  45780237       287476      Natural causes        NaT       Auto   
828  89833962       441097        Other causes        NaT       Auto   
505  35782742       524545  Other driver error        NaT       Auto   

    police_report           claim_type  claim_amount  total_policy_claims  \
184            No          Injury only           NaN                  1.0   
836            No  Material and injury       39192.0                  1.0   
677       Unknown        Material only        1621.5                  2.0   
828           Yes  Material and injury       37040.0                  1.0   
505            No          Injury only       35250.0                  3.0   

    fraudulent   CUST_ID  gender DateOfBirth State       Contact   Segment  
184        Yes  154557.0  F